In [5]:
!pip install fastapi uvicorn joblib pandas scikit-learn shap pyngrok

In [6]:
import pandas as pd
import os

DATASET_DIR = "datasets/"

def load_multiple_datasets():
    datasets = {}
    for filename in os.listdir(DATASET_DIR):
        if filename.endswith(".csv"):
            file_path = os.path.join(DATASET_DIR, filename)
            df = pd.read_csv(file_path)
            datasets[filename] = df
    return datasets


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

def preprocess_data(df):
    if 'customerID' in df.columns:
        df = df.drop('customerID', axis=1)

    for col in df.select_dtypes(include=['object']).columns:
        df[col] = LabelEncoder().fit_transform(df[col])

    X = df.drop('Churn', axis=1)
    y = df['Churn']
    return X, y

def train_model_on_dataset(df):
    X, y = preprocess_data(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")

    return model


In [8]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib

class ChurnInput(BaseModel):
    tenure: int
    MonthlyCharges: float
    TotalCharges: float
    gender: str
    SeniorCitizen: int
    Partner: str
    Dependents: str
    PhoneService: str
    MultipleLines: str
    InternetService: str
    OnlineSecurity: str
    OnlineBackup: str
    DeviceProtection: str
    TechSupport: str
    StreamingTV: str
    StreamingMovies: str
    Contract: str
    PaperlessBilling: str
    PaymentMethod: str

app = FastAPI()

def preprocess_input(data):
    df = pd.DataFrame([data.dict()])
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = LabelEncoder().fit_transform(df[col])
    return df

@app.post("/predict/")
def predict_churn(input_data: ChurnInput):

    model = joblib.load("churn_model.pkl")

    input_df = preprocess_input(input_data)

    prediction = model.predict(input_df)
    prediction


In [22]:
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os

DATASET_DIR = "/content/drive/MyDrive/data_sets"

def load_and_preprocess_dataset(filename):
    file_path = os.path.join(DATASET_DIR, filename)
    df = pd.read_csv(file_path)
    if 'customerID' in df.columns:
        df = df.drop('customerID', axis=1)
    print(df.columns)
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = LabelEncoder().fit_transform(df[col])
    X = df.drop('churn', axis=1)
    y = df['churn']
    return X, y
def train_model_on_dataset(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    return model
X, y = load_and_preprocess_dataset('/content/drive/MyDrive/data_sets/Bank_churn.csv')
model = train_model_on_dataset(X,y)
explainer = shap.TreeExplainer(model)
@app.post("/explain/")
def explain_churn(input_data: ChurnInput):
    pass

Index(['rownumber', 'customerid', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
       'isactivemember', 'estimatedsalary', 'churn'],
      dtype='object')


In [23]:
import shap
explainer = shap.TreeExplainer(model)

@app.post("/explain/")
def explain_churn(input_data: ChurnInput):
    input_df = preprocess_input(input_data)
    shap_values = explainer.shap_values(input_df)
    explanation = shap_values[1].tolist()

    return {"explanation": explanation}


In [26]:
!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.5 MB/s eta 0:00:00


In [27]:
# Install ngrok to tunnel the FastAPI server
!pip install pyngrok

# Import and set up ngrok
from pyngrok import ngrok

# Set up FastAPI server
public_url = ngrok.connect(port='8000')
print(f"FastAPI public URL: {public_url}")

# Run FastAPI with Uvicorn
!uvicorn api:app --host 0.0.0.0 --port 8000


ERROR:pyngrok.process.ngrok:t=2024-09-19T04:42:25+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-09-19T04:42:25+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.